```python
# input_data shape
Input: (batch_size, in_channel, width, height)
# conv layer
class torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True)
input: (Batch_size, C_in, H_in, W_in)
output: (Batch_size, C_out, H_out, W_out)

weight(tensor): (out_channels, in_channels,kernel_size)
bias(tensor): (out_channel)
```

In [9]:
from itertools import product

import time
import numpy as np
from sklearn import datasets
from scipy.stats import multivariate_normal as mvn
import matplotlib.pyplot as plt

from ripser import Rips
from persim import PersistenceImager

import os
import math

import seaborn
import pandas as pd
import torch
import random
from torch import nn
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import persim

import re
import cv2
from PIL import Image
import datetime

In [10]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
random.seed(111)
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)
# parameters
learning_rate = 0.001
training_epochs = 100
batchsize = 25
testbatchsize = 10


In [11]:
# dir = os.listdir("./metric")
# data = []
# a = []
# for metric in dir:
#     a = re.findall("\d+\.?\d*", metric)
#     data.append([np.loadtxt("./metric/"+metric), metric])
#     # print(a[-2])
#     print(metric[-2])


In [12]:
from operator import index




In [13]:
class My_dataset(Dataset):
    def __init__(self, train, index=range(0,100)):
        super().__init__()
        # added_data = ""
        added_data = "/mnt/sda3/docker_space/Code/TDA-NN/3090/grid_save/pattern04-17--16-10-02/"
        if train:
            path = "/mnt/sda3/docker_space/Code/TDA-NN/3090/grid_save/pattern04-17--16-17-55/"
            dir_data = os.listdir(path)
            # added_data = os.listdir("/mnt/sda3/docker_space/Code/TDA-NN/3090/grid_save/model_mnist_Nov.25_19.38.54")
            # added_path = "/mnt/sda3/docker_space/Code/TDA-NN/3090/grid_save/model_mnist_Nov.25_19.38.54/"
        else:
            path = "/mnt/sda3/docker_space/Code/TDA-NN/3090/grid_save/pattern04-18--15-16-00/"
            dir_data = os.listdir(path)
        data = []
        sample = []
        distance = 1
        T_normal = []
        T_attack = []
        normal_attack = [0,0]
        for metric in dir_data[0:4]:
            dgm = []
            grid = pd.DataFrame(np.loadtxt(path+metric))
            for i in range(grid.shape[0]):
                for j in range(grid.shape[1]):
                    if grid[i][j] > 0:
                        dgm.append([i,j])
            T_normal.append(dgm)
        for metric in dir_data[-4:]:
            dgm = []
            grid = pd.DataFrame(np.loadtxt(path+metric))
            for i in range(grid.shape[0]):
                for j in range(grid.shape[1]):
                    if grid[i][j] > 0:
                        dgm.append([i,j])
            T_attack.append(dgm)
            
        for metric in dir_data:
            if int(re.findall("\d+\.?\d*", metric)[0]) in index:
                sample.append([np.loadtxt(path+metric), metric])
                label = int("attack" in metric)

                # if (label == 0):
                #     distance_bottleneck, matching = persim.bottleneck(sample[len(sample)-1][0], T_normal[len(sample)-1], matching=True)
                # else:
                #     distance_bottleneck, matching = persim.bottleneck(sample[len(sample)-1][0], T_attack[len(sample)-1], matching=True)
                # distance += distance_bottleneck
                if len(sample) == 4:
                    res = cv2.merge([i[0] for i in sample])
                    res = np.transpose(res,(2,0,1))
                    # d = 1/(1+math.exp(math.log(distance)))
                    data.append([res, np.array((label,distance))])
                    sample = []

        if added_data:
            dir_data = os.listdir(added_data)
            for metric in dir_data:
                if int(re.findall("\d+\.?\d*", metric)[0]) in index:
                    sample.append([np.loadtxt(added_data+metric), metric])
                    label = int("attack" in metric)

                    # if (label == 0):
                    #     distance_bottleneck, matching = persim.bottleneck(sample[len(sample)-1][0], T_normal[len(sample)-1], matching=True)
                    # else:
                    #     distance_bottleneck, matching = persim.bottleneck(sample[len(sample)-1][0], T_attack[len(sample)-1], matching=True)
                    # distance += distance_bottleneck
                    if len(sample) == 4:
                        res = cv2.merge([i[0] for i in sample])
                        res = np.transpose(res,(2,0,1))
                        # d = 1/(1+math.exp(math.log(distance)))
                        data.append([res, np.array((label,distance))])
                        sample = []
                        
        data.sort(key=lambda x: x[1][0])
        if train:
            for i in data:
                normal_attack[int(i[1][0])] += 1
            print(normal_attack)
            normal_attack = [max(normal_attack), min(normal_attack)]
            data.sort(key=lambda x: x[1][0])
            data = data[0:normal_attack[1]]+data[-normal_attack[1]:]
            normal_attack = [0,0]
            for i in data:
                normal_attack[int(i[1][0])] += 1
            print(normal_attack)
        
        self.x = [item[0] for item in data]
        self.y = [item[1] for item in data]
        # self.y = [int(re.findall("\d+\.?\d*", item[1])[0]) for item in data]
        self.src,  self.trg = [], []
        

        for i in range(len(data)):
            self.src.append(self.x[i])
            self.trg.append(self.y[i])

    def __getitem__(self, index):
        return self.src[index], self.trg[index]

    def __len__(self):
        return len(self.src)

 # 或者return len(self.trg), src和trg长度一样


In [14]:
data_tf = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize([0.5], [0.5])])
     
data_train = My_dataset(train=True, index=range(10,45))
data_test = My_dataset(train=False, index=range(10,45))
data_loader_train = DataLoader(data_train, batch_size=batchsize, shuffle=True)
data_loader_test = DataLoader(data_test, batch_size=testbatchsize, shuffle=False)


# i_batch的多少根据batch size和def __len__(self)返回的长度确定
# batch_data返回的值根据def __getitem__(self, index)来确定
# 对训练集：(不太清楚enumerate返回什么的时候就多print试试)
print("train")
for i_batch, batch_data in enumerate(data_loader_train):
    print(i_batch)  # 打印batch编号
    print(batch_data[0].shape)  # 打印该batch里面src
    print(batch_data[1])  # 打印该batch里面trg
# # 对测试集：（下面的语句也可以）
print("test")
for i_batch, batch_data in enumerate(data_loader_test):
    print(i_batch)  # 打印batch编号
    print(batch_data[0].shape)  # 打印该batch里面src
    print(batch_data[1])  # 打印该batch里面trg



[0, 960]
[0, 960]
train
0
torch.Size([25, 4, 128, 128])
tensor([[1, 1],
        [1, 1],
        [1, 1],
        [1, 1],
        [1, 1],
        [1, 1],
        [1, 1],
        [1, 1],
        [1, 1],
        [1, 1],
        [1, 1],
        [1, 1],
        [1, 1],
        [1, 1],
        [1, 1],
        [1, 1],
        [1, 1],
        [1, 1],
        [1, 1],
        [1, 1],
        [1, 1],
        [1, 1],
        [1, 1],
        [1, 1],
        [1, 1]])
1
torch.Size([25, 4, 128, 128])
tensor([[1, 1],
        [1, 1],
        [1, 1],
        [1, 1],
        [1, 1],
        [1, 1],
        [1, 1],
        [1, 1],
        [1, 1],
        [1, 1],
        [1, 1],
        [1, 1],
        [1, 1],
        [1, 1],
        [1, 1],
        [1, 1],
        [1, 1],
        [1, 1],
        [1, 1],
        [1, 1],
        [1, 1],
        [1, 1],
        [1, 1],
        [1, 1],
        [1, 1]])
2
torch.Size([25, 4, 128, 128])
tensor([[1, 1],
        [1, 1],
        [1, 1],
        [1, 1],
        [1, 1]

In [15]:
class MyNet(torch.nn.Module):
    def __init__(self):
        super(MyNet, self).__init__()
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(4, 16, kernel_size=64),
            torch.nn.BatchNorm2d(16),
            torch.nn.ReLU(inplace=True)
        )

        self.layer2 = torch.nn.Sequential(
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.fc = torch.nn.Sequential(
            torch.nn.Linear(16 * 32 * 32, 1024),
            torch.nn.ReLU(inplace=True),
            torch.nn.Linear(1024, 1024),
            torch.nn.ReLU(inplace=True),
            torch.nn.Linear(1024, 128),
            torch.nn.ReLU(inplace=True),
            torch.nn.Linear(128, 64),
            torch.nn.ReLU(inplace=True),
            torch.nn.Linear(64, 2)
        )

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x



In [16]:
class CustomLoss2(nn.Module):
    def __init__(self) -> None:
        super().__init__()
    def forward(self, output, target):
        res = -output.gather(dim=1, index=target.view(-1, 1))
        res += torch.log(torch.exp(output).sum(dim=1).view(-1, 1))
        res = res.mean()
        return res
    def forward(self, output, target, distance):
        res = -output.gather(dim=1, index=target.view(-1, 1))
        res += torch.log(torch.exp(output).sum(dim=1).view(-1, 1))
        res = res.mean() * distance.mean()
        return res
        

In [17]:
from typing import Callable, Optional
from torch.nn import functional as F
from torch.nn import _reduction as _Reduction
from torch import Tensor
class _Loss(nn.Module):
    reduction: str

    def __init__(self, size_average=None, reduce=None, reduction: str = 'mean') -> None:
        super(_Loss, self).__init__()
        if size_average is not None or reduce is not None:
            self.reduction: str = _Reduction.legacy_get_string(size_average, reduce)
        else:
            self.reduction = reduction
class _WeightedLoss(_Loss):
    def __init__(self, weight: Optional[Tensor] = None, size_average=None, reduce=None, reduction: str = 'mean') -> None:
        super(_WeightedLoss, self).__init__(size_average, reduce, reduction)
        self.register_buffer('weight', weight)
        self.weight: Optional[Tensor]
class CELoss(_WeightedLoss): # 注意继承 nn.Module
    __constants__ = ['ignore_index', 'reduction', 'label_smoothing']
    ignore_index: int
    label_smoothing: float

    def __init__(self, weight: Optional[Tensor] = None, size_average=None, ignore_index: int = -100,
                 reduce=None, reduction: str = 'mean', label_smoothing: float = 0.0) -> None:
        super(CELoss, self).__init__(weight, size_average, reduce, reduction)
        self.ignore_index = ignore_index
        self.label_smoothing = label_smoothing

    def forward(self, input: Tensor, target: Tensor, distance) -> Tensor:
        return distance.mean() * F.cross_entropy(input, target, weight=self.weight,
                               ignore_index=self.ignore_index, reduction=self.reduction,
                               label_smoothing=self.label_smoothing)

In [18]:

model = MyNet().to(device)
# define cost/loss & optimizer
# Softmax is internally computed.
# criterion = CELoss().to(device)
criterion = CELoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)


In [19]:
from copy import deepcopy


total_batch = len(data_loader_train)
top = 0
for epoch in range(training_epochs):
    model.train()
    avg_cost = 0

    # for X, Y in data_loader_train:
    #     X = torch.autograd.Variable(X).to(device).float()
    #     Y = Y.to(device)

    #     optimizer.zero_grad()
    #     hypothesis = model(X)
    #     cost = criterion(hypothesis, Y)
    #     cost.backward()
    #     optimizer.step()

    #     avg_cost += cost / total_batch
    # model.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(data_loader_train):
        distance = targets[:,1].to(device).float()
        targets = targets[:,0]
        inputs, targets = inputs.to(device).float(), targets.to(device).to(torch.int64)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets, distance)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        avg_cost += loss / total_batch

    
    # if avg_cost < 0.1:
    #     break
    model.eval()
    # with torch.no_grad():
    #     for X_test, Y_test in data_loader_test:
    #         X_test = X_test.to(device).float()
    #         Y_test = Y_test.to(device)

    #     prediction = model(X_test)
    #     correct_prediction = torch.argmax(prediction, 1) == Y_test
    #     accuracy = correct_prediction.float().mean()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(data_loader_train):
            distance = targets[:,1].to(device).float()
            targets = targets[:,0]
            inputs, targets = inputs.to(device).float(), targets.to(device).to(torch.int64)
            outputs = model(inputs)
            loss = criterion(outputs, targets, distance)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    trainacc = 100.*correct/total
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(data_loader_test):
            distance = targets[:,1].to(device).float()
            targets = targets[:,0]
            inputs, targets = inputs.to(device).float(), targets.to(device).to(torch.int64)
            outputs = model(inputs)
            loss = criterion(outputs, targets, distance)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    testacc = 100.*correct/total

    if testacc > top and epoch > 50:
        top = testacc
        topmodel = deepcopy(model)
    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost), 'trainacc:', trainacc, 'testacc:', testacc, 'test_loss:', test_loss)

print('Learning finished')


Epoch: 0001 cost = 0.593615174 trainacc: 100.0 testacc: 89.38271604938272 test_loss: 47.73193711042404
Epoch: 0002 cost = 0.523931801 trainacc: 100.0 testacc: 89.38271604938272 test_loss: 43.160527884960175
Epoch: 0003 cost = 0.435937256 trainacc: 100.0 testacc: 89.38271604938272 test_loss: 37.35401725769043
Epoch: 0004 cost = 0.317149967 trainacc: 100.0 testacc: 89.38271604938272 test_loss: 31.570758685469627
Epoch: 0005 cost = 0.195952579 trainacc: 100.0 testacc: 89.38271604938272 test_loss: 28.461644411087036
Epoch: 0006 cost = 0.112956852 trainacc: 100.0 testacc: 89.38271604938272 test_loss: 28.8330565020442
Epoch: 0007 cost = 0.068709396 trainacc: 100.0 testacc: 89.38271604938272 test_loss: 30.988678507506847
Epoch: 0008 cost = 0.046073351 trainacc: 100.0 testacc: 89.38271604938272 test_loss: 34.030100505799055
Epoch: 0009 cost = 0.033560064 trainacc: 100.0 testacc: 89.38271604938272 test_loss: 36.30608099978417
Epoch: 0010 cost = 0.026500979 trainacc: 100.0 testacc: 89.3827160493

In [20]:
model = deepcopy(topmodel)
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(data_loader_test):
        # if batch_idx >= 34: break
        # if batch_idx < 20: continue
        distance = targets[:,1].to(device).float()
        targets = targets[:,0]
        inputs, targets = inputs.to(device).float(), targets.to(device).to(torch.int64)
        outputs = model(inputs)
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        print(torch.argmax(outputs, 1), targets)
    testacc = 100.*correct/total
    print('testacc:', testacc)

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0') tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0') tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0') tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0') tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0') tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0') tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0') tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0') tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0') tensor([0, 0, 0, 0, 0, 0, 1, 1, 1, 1], device='c

In [21]:
savepath = "./classifier/single="
now = datetime.datetime.now()
state = {'state_dict': model.state_dict(), 'optimizer': optimizer.state_dict(), 'epoch': epoch}
print(savepath+now.strftime("%m-%d--%H-%M-%S")+".pth")
# torch.save(state, savepath+now.strftime("%m-%d--%H-%M-%S")+".pth")

./classifier/single=04-19--21-07-45.pth
